In [1]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from cdp_langchain.agent_toolkits import CdpToolkit
from cdp_langchain.utils import CdpAgentkitWrapper


In [2]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

True

In [3]:
cdp= CdpAgentkitWrapper()
toolkit = CdpToolkit.from_cdp_agentkit_wrapper(cdp)


In [7]:
type(toolkit)

cdp_langchain.agent_toolkits.cdp_toolkit.CdpToolkit

In [30]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(toolkit.get_tools()[1])

CdpTool(name='deploy_contract', description='\nDeploys smart contract with required args: solidity version (string), solidity input json (string), contract name (string), and optional constructor args (Dict[str, Any])\n\nInput json structure:\n{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{}}\n\nYou must set the outputSelection to {"*":{"*":["abi","evm.bytecode"]}} in the settings. The solidity version must be >= 0.8.0 and <= 0.8.28.\n\nSources should contain one or more contracts with the following structure:\n{"contract_name.sol":{"content":"contract code"}}\n\nThe contract code should be escaped. Contracts cannot import from external contracts but can import from one another.\n\nConstructor args are required if the contract has a constructor. They are a key-value\nmap where the key is the arg name and the value is the arg value. Encode uint/int/bytes/string/address values as strings, boolean values as true/false. F

In [42]:
toolkit.get_tools()[1].invoke( input={
    "language":"Solidity",
    "settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},
    "sources":{
        "contracts/Verified.sol": {
        "content": "//SPDX-License-Identifier: MIT\r\npragma solidity ^0.8.18;\r\n\r\ncontract Verified {\r\n    string public greet =\r\n        \"Increase this counter if you completed this tutorial\";\r\n    string public tutorial =\r\n        \"https://docs.etherscan.io/contract-verification/multichain-verification\";\r\n\r\n    uint256 public verified = 0;\r\n\r\n    function completedTutorial() public {\r\n        verified += 1;\r\n    }\r\n}\r\n"
      }
    }
}

)

ValidationError: 3 validation errors for DeployContractInput
solidity_version
  Field required [type=missing, input_value={'language': 'Solidity', ...1;\r\n    }\r\n}\r\n'}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
solidity_input_json
  Field required [type=missing, input_value={'language': 'Solidity', ...1;\r\n    }\r\n}\r\n'}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
contract_name
  Field required [type=missing, input_value={'language': 'Solidity', ...1;\r\n    }\r\n}\r\n'}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [23]:
toolkit.get_tools()[1].invoke(input={'solidity_version': '0.8.26', 'solidity_input_json': '{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{"contracts/Calculator.sol":{"content":"// SPDX-License-Identifier: MIT\\n\\npragma solidity ^0.8.26;\\n\\ncontract Calculator {\\n\\n    uint256 result = 0;\\n\\n    function add(uint256 num) public {\\n        result += num;\\n    }\\n\\n    function subtract(uint256 num) public {\\n        result -= num;\\n    }\\n\\n    function multiply(uint256 num) public {\\n        result *= num;\\n    }\\n\\n    function get() public view returns (uint256) {\\n        return result;\\n    }\\n}\\n"}}}', 'contract_name': 'Calculator', })
                              

'Deployed contract Calculator at address 0x7f475E385fbf4ac58734b352f5532bCb509d290a. Transaction link: https://sepolia.basescan.org/tx/0x03cd6cc4732ea81192560bce08fb8b58ed9abf913841fe4aa148d5786fb83a97'

In [60]:
toolkit.get_tools()[1].args_schema.model_json_schema()

{'description': 'Input argument schema for deploy contract action.', 'properties': {'solidity_version': {'description': 'The solidity compiler version', 'title': 'Solidity Version', 'type': 'string'}, 'solidity_input_json': {'description': 'The input json for the solidity compiler', 'title': 'Solidity Input Json', 'type': 'string'}, 'contract_name': {'description': 'The name of the contract class to be deployed', 'title': 'Contract Name', 'type': 'string'}, 'constructor_args': {'anyOf': [{'type': 'object'}, {'type': 'null'}], 'default': None, 'description': 'The constructor arguments for the contract', 'title': 'Constructor Args'}}, 'required': ['solidity_version', 'solidity_input_json', 'contract_name'], 'title': 'DeployContractInput', 'type': 'object'}

In [4]:
from langgraph.prebuilt import create_react_agent
tools = toolkit.get_tools()
llm = ChatOpenAI(model='gpt-4o-mini')
react_agent = create_react_agent(llm, tools)

In [6]:
from langchain_core.messages import HumanMessage

react_agent.invoke({"messages":[HumanMessage(content="do you have access to any tools to help deploy smart contracts onto the blockchain?")]})

{'messages': [HumanMessage(content='do you have access to any tools to help deploy smart contracts onto the blockchain?', additional_kwargs={}, response_metadata={}, id='cbaab945-aaf6-42e8-9df9-f82e26c68d01'),
  AIMessage(content='Yes, I have the ability to deploy smart contracts onto the blockchain. If you provide me with the details such as the Solidity version, contract code, contract name, and any constructor arguments, I can assist you in deploying the contract.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 4382, 'total_tokens': 4431, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 4224}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e6dfccc-5d8e-40ef-846a-22f7d3aca75e-0', usa

In [21]:
react_agent.invoke({"messages":[HumanMessage(content="""please assist deploying a contract named Calculator, using solidity version 0.8.26 using the following contract code: "solidity_input_json":{"language":"Solidity",
                                                            "settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},
                                                            "sources":{"contracts/Verified.sol": {
                                                                "content": "// SPDX-License-Identifier: MIT\n\npragma solidity ^0.8.26;\n\ncontract Calculator {\n\n    uint256 result = 0;\n\n    function add(uint256 num) public {\n        result += num;\n    }\n\n    function subtract(uint256 num) public {\n        result -= num;\n    }\n\n    function multiply(uint256 num) public {\n        result *= num;\n    }\n\n    function get() public view returns (uint256) {\n\n        return result;\n    }\n\n}\n"}
                                                            }
                                                            }""")]})

{'messages': [HumanMessage(content='please assist deploying a contract named Calculator, using solidity version 0.8.26 using the following contract code: "solidity_input_json":{"language":"Solidity",\n                                                            "settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},\n                                                            "sources":{"contracts/Verified.sol": {\n                                                                "content": "// SPDX-License-Identifier: MIT\n\npragma solidity ^0.8.26;\n\ncontract Calculator {\n\n    uint256 result = 0;\n\n    function add(uint256 num) public {\n        result += num;\n    }\n\n    function subtract(uint256 num) public {\n        result -= num;\n    }\n\n    function multiply(uint256 num) public {\n        result *= num;\n    }\n\n    function get() public view returns (uint256) {\n\n        return result;\n    }\n\n}\n"}\n                                         

In [15]:
toolkit.get_tools()[4].invoke(input={"asset_id":"eth"})

'Balances for wallet 974f8dbf-b045-4058-9d2a-4d10121e1140:\n  0x98750500e358fAa19a78c3f315DfC57eAf32Ce8a: 0.0001'

In [20]:
toolkit.get_tools()[13].invoke(input={"asset_id":"eth",
                                      "wallet":"0x98750500e358fAa19a78c3f315DfC57eAf32Ce8a"})

'Received eth from the faucet. Transaction: https://sepolia.basescan.org/tx/0xc415993522d7782d040be8a850705ce33c02fb4621974bf9f787744112ca9cd6'

In [24]:
contract_template = {'solidity_version': '0.8.26', 'solidity_input_json': '{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{"contracts/Calculator.sol":{"content":"// SPDX-License-Identifier: MIT\\n\\npragma solidity ^0.8.26;\\n\\ncontract Calculator {\\n\\n    uint256 result = 0;\\n\\n    function add(uint256 num) public {\\n        result += num;\\n    }\\n\\n    function subtract(uint256 num) public {\\n        result -= num;\\n    }\\n\\n    function multiply(uint256 num) public {\\n        result *= num;\\n    }\\n\\n    function get() public view returns (uint256) {\\n        return result;\\n    }\\n}\\n"}}}', 'contract_name': 'Calculator'}

In [25]:
contract_template

{'solidity_version': '0.8.26',
 'solidity_input_json': '{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{"contracts/Calculator.sol":{"content":"// SPDX-License-Identifier: MIT\\n\\npragma solidity ^0.8.26;\\n\\ncontract Calculator {\\n\\n    uint256 result = 0;\\n\\n    function add(uint256 num) public {\\n        result += num;\\n    }\\n\\n    function subtract(uint256 num) public {\\n        result -= num;\\n    }\\n\\n    function multiply(uint256 num) public {\\n        result *= num;\\n    }\\n\\n    function get() public view returns (uint256) {\\n        return result;\\n    }\\n}\\n"}}}',
 'contract_name': 'Calculator'}

In [ ]:
def complete_contract_template(num:int):
    contract_template = {'solidity_version': '0.8.26', 'solidity_input_json': '{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{"contracts/Calculator.sol":{"content":f"// SPDX-License-Identifier: MIT\\n\\npragma solidity ^0.8.26;\\n\\ncontract Calculator {\\n\\n    uint256 result = 0;\\n\\n    function add(uint256 num) public {\\n        result += num;\\n    }\\n\\n    function subtract(uint256 num) public {\\n        result -= num;\\n    }\\n\\n    function multiply(uint256 num) public {\\n        result *= num;\\n    }\\n\\n    function get() public view returns (uint256) {\\n        return result;\\n    }\\n}\\n"}}}', 'contract_name': 'Calculator','constructor_args': '{"param":{num}}

In [43]:
num = '5'
toolkit.get_tools()[1].invoke(input={'solidity_version': '0.8.26', 'solidity_input_json': '{"language":"Solidity","settings":{"remappings":[],"outputSelection":{"*":{"*":["abi","evm.bytecode"]}}},"sources":{"contracts/Calculator.sol":{"content":"// SPDX-License-Identifier: MIT\\n\\npragma solidity ^0.8.26;\\n\\ncontract Calculator{\\n\\n    uint256 result; \\n\\n    constructor(uint256 _result) {\\n        result = _result;\\n    }\\n\\n    \\n    \\n\\n    // Frequency of contributions is based on a weekly cycle. \\n    // A week contains 7 days so if the frequency is set to 1 then it will be once every week. If you wanted for example to make 3 weeks then we would have set frequency to 2, and there would be one contribution made every other week as opposed to three contributions made every week.\\n    // Note: There are no restrictions on how many times a user can contribute. This is just an implementation detail. We will allow the contract owner to change this in future releases of the dApp so that users can control their\\n    \\n\\n\\n    function add(uint256 num) public returns(uint256) {\\n        result += num;\\n\\n        return result;\\n    }\\n\\n    function get() public view returns(uint256) {\\n\\n        return result;\\n    }\\n\\n\\n}"}}}', 'contract_name': 'Calculator', 'constructor_args': {"_result":f"{num}"}})

'Deployed contract Calculator at address 0x072C26508cF521071495db565153eBae8a75d61e. Transaction link: https://sepolia.basescan.org/tx/0x9e9e139429ca3b5286c1effd29f1bfc68e4c0cb89c842ee0f86ff0f50e5851f8'

In [47]:
react_agent.invoke({"messages": "What was my last question?", "chat_history":"HumanMessage(content=\"What can you assist with?\"), AIMessage(content=\"I can assist you with a variety of tasks related to blockchain and cryptocurrency, including:\n\n1. **Wallet Management**:\n   - Check wallet balances for different assets.\n   - Get details about your wallet.\n\n2. **Token Management**:\n   - Deploy ERC-20 tokens and ERC-721 NFTs.\n   - Transfer tokens and NFTs between addresses.\n   - Mint new NFTs.\n\n3. **Trading and Transactions**:\n   - Trade assets on supported networks.\n   - Wrap ETH to WETH.\n\n4. **Smart Contract Deployment**:\n   - Deploy custom smart contracts using Solidity.\n\n5. **Reputation Checks**:\n   - Check the reputation of Ethereum addresses on specific networks.\n\n6. **Fund Requests**:\n   - Request test tokens from faucets (for test networks).\n\n7. **Superfluid Money Flow**:\n   - Create, update, and delete money flows using Superfluid.\n\n8. **Price Fetching**:\n   - Fetch token prices from Pyth price feeds.\n\n9. **Basename Registration**:\n   - Register a Basename for your wallet.\n\nIf you have a specific task in mind, feel free to ask!'\")"})

InvalidUpdateError: Must write to at least one of ['messages', 'structured_response']

In [61]:
from langchain import hub

prompt = hub.pull("hwchase17/react")

/Users/pat/PycharmProjects/eth_tanda/.venv/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [50]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat', 'lc_hub_commit_hash': '3ecd5f710db438a9cf3773c57d6ac8951eefd2cd9a9b2a0026a65a0893b86a6e'}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative res

In [66]:
react_agent_w_history =  create_react_agent(llm, tools, state_modifier=prompt)

In [67]:
react_agent_w_history.invoke(
    {
        "input": "hi"
    })
            

InvalidUpdateError: Must write to at least one of ['messages', 'structured_response']

In [94]:
from langchain_core.prompts import ChatPromptTemplate
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "A tanda is a regional version of a rotating savings and credit association (ROSCA). It is a form of a short-term no-interest loan among a group of friends and family where they contribute money to a pool that is then rotated between the members. As an example, a tanda is formed between ten friends and family. Each member gives $100 USD every two weeks to the group's organizer. At the end of the month, one participant gets the pot, $2000. Or, each member gives $100 weekly, and each week one of the ten participants get the pot ($1,000). This continues until each member has received the pot. This concept is being adapted to blockchain technology for decentralized processing. You are an expert on this service and how smart contracts work.\n \n You are a customer service agent assigned to assist clients by collecting pertinent information about how their group would like to structure their Etherium Tanda association, and answer any questions they may have about tandas or smart contracts.Answer client questions and collect all information required to structure their Tanda association in the form of a ETH smart contract. This includes the total number of members in the group, how frequently contributions will be collected, amount of member contributions, at what interval will the pot be distributed, and all ETH cryptocurrency wallet addresses of members participating in the Tanda association. If any information is missing please ask the client for the information, and let them what information is needed.\n Use the conversation history below to respond to questions conversationally.\n {chat_history}"),
    ('user', "{input}"),
    ("placeholder", "{agent_scratchpad}")])



In [78]:
tanda_agent.invoke({'messages': 'what can you help me with?', "input":"and how might that benefit me?"})

{'messages': [HumanMessage(content='what can you help me with?', additional_kwargs={}, response_metadata={}, id='df113364-be46-468a-b761-11835e218853'),
  AIMessage(content="Hello! I'm here to help you set up your Ethereum Tanda association and answer any questions you may have about tandas or smart contracts. \n\nTo get started, could you please provide some details about how your group would like to structure the Tanda? Here are a few questions to consider:\n\n1. **Group Size**: How many members will be participating in the Tanda?\n2. **Contribution Amount**: How much will each member contribute to the Tanda?\n3. **Frequency**: How often will contributions be made (e.g., weekly, bi-weekly)?\n4. **Duration**: How long will the Tanda run (e.g., for a specific number of cycles or months)?\n5. **Payout Structure**: How will payouts be structured? Will it rotate among members or follow a different pattern?\n6. **Smart Contract Features**: Are there any specific features you would like in 

In [73]:
tanda_agent

AttributeError: 'CompiledStateGraph' object has no attribute 'prompt'

In [95]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
    
agent = create_tool_calling_agent(llm, tools=tools, prompt=prompt2)

tanda_agent2 = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [102]:
result = tanda_agent2.invoke({'input': "What information do you need from me to set one up?", 'chat_history':"""'input': 'How would that work on the blockchain?',
 'chat_history': '\'input\': \'What is a tanda?\',\n \'chat_history\': \' \',\n \'output\': \'A tanda is a regional version of a rotating savings and credit association (ROSCA). It is a financial arrangement typically formed among a group of friends or family members who contribute money to a common pool. Each member contributes a set amount of money at regular intervals, and the total amount collected is then rotated among the members, allowing each person to receive the pot at different times.\n\nFor example, in a tanda with ten members, each member might contribute $100 every two weeks. At the end of the month, one member would receive the total pot of $2,000. This continues until each member has received the pot.\n\nTandas can be a helpful way for individuals to save money or access short-term funds without interest, as the contributions are returned to the members in a structured manner. The concept is being adapted to blockchain technology for decentralized processing, allowing for more transparency and efficiency in managing these arrangements.\' ",\n \'output\': "Tandas, or rotating savings and credit associations (ROSCA), offer several advantages:\n\n1. **Access to Funds**: Members can access a lump sum of money when it\'s their turn to receive the pot, which can be helpful for large purchases or emergencies.\n\n2. **No Interest**: Tandas do not involve interest payments, making them a cost-effective option for borrowing money compared to traditional loans.\n\n3. **Encourages Saving**: Regular contributions can help members save money over time, as they commit to putting aside a specific amount regularly.\n\n4. **Community Support**: Tandas foster a sense of community and trust among members, as they depend on each other to make the arrangement work.\n\n5. **Financial Discipline**: The structure of regular contributions encourages members to develop financial discipline and budgeting skills.\n\n6. **Flexibility**: Tandas can be customized to fit the needs and preferences of the group, including contribution amounts and frequencies.\n\n7. **Decentralization and Transparency**: With the adaptation of blockchain technology, tandas can offer increased transparency and decentralized management, reducing the risk of mismanagement or fraud.\n\n8. **Cultural Relevance**: Tandas hold cultural significance in many communities, making them a familiar and trusted financial tool. \n\nThese advantages make tandas an appealing option for individuals looking to manage their finances collaboratively and effectively." ',
 'output': "On the blockchain, a tanda operates through a smart contract, which is a self-executing contract with the terms of the agreement directly written into code. Here's how it would work:\n\n1. **Smart Contract Creation**: A smart contract is deployed on the Ethereum blockchain to manage the tanda. This contract will outline the terms, including the number of participants, contribution amounts, and the schedule for pot distribution.\n\n2. **Member Registration**: Participants in the tanda must register their Ethereum wallet addresses with the smart contract. This ensures that all transactions are tracked and managed transparently.\n\n3. **Contribution Collection**: At the agreed-upon intervals (e.g., weekly or bi-weekly), the smart contract will automatically collect contributions from each member's wallet. The contract can enforce rules to ensure that contributions are made on time.\n\n4. **Pot Distribution**: After collecting contributions, the smart contract will distribute the total pot to the designated member based on the pre-defined schedule. For example, if the pot is to be distributed weekly, the smart contract will determine which member receives the pot each week.\n\n5. **Transparency and Security**: All transactions and contributions are recorded on the blockchain, providing full transparency to all members. This reduces the risk of mismanagement or fraud, as the smart contract executes actions without human intervention.\n\n6. **Final Settlement**: Once all members have received their turn with the pot, the smart contract can either be terminated or set to a new cycle for future contributions and distributions.\n\nThis decentralized approach ensures fairness and accountability, as the process is automated and visible to all participants, leveraging the security and transparency features of blockchain technology. If you have specific questions or need to structure your own tanda on the blockchain, I can help with that!" """})



> Entering new AgentExecutor chain...
To set up your Ethereum Tanda association, I will need the following information:

1. **Total Number of Members**: How many participants will be in the Tanda?
2. **Frequency of Contributions**: How often will members contribute (e.g., weekly, bi-weekly)?
3. **Contribution Amount**: How much will each member contribute per interval (e.g., $100 USD)?
4. **Pot Distribution Interval**: At what interval will the total pot be distributed (e.g., weekly, monthly)?
5. **Ethereum Wallet Addresses**: Please provide the Ethereum wallet addresses of all members participating in the Tanda.

Once I have this information, I can assist you in structuring the Tanda association as a smart contract. If you have any questions about any of these points, feel free to ask!

> Finished chain.


In [105]:
result.keys()

dict_keys(['input', 'chat_history', 'output'])

In [106]:
result['output']

'To set up your Ethereum Tanda association, I will need the following information:\n\n1. **Total Number of Members**: How many participants will be in the Tanda?\n2. **Frequency of Contributions**: How often will members contribute (e.g., weekly, bi-weekly)?\n3. **Contribution Amount**: How much will each member contribute per interval (e.g., $100 USD)?\n4. **Pot Distribution Interval**: At what interval will the total pot be distributed (e.g., weekly, monthly)?\n5. **Ethereum Wallet Addresses**: Please provide the Ethereum wallet addresses of all members participating in the Tanda.\n\nOnce I have this information, I can assist you in structuring the Tanda association as a smart contract. If you have any questions about any of these points, feel free to ask!'